In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random
from sklearn.manifold import TSNE
import seaborn as sns
import plotly.express as px
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.manifold import TSNE
import seaborn as sns
import plotly.express as px

In [26]:
df = pd.read_csv("phoneprices.csv")

x = df.iloc[:,:-1]

x_normalized = (x - x.min()) / (x.max() - x.min())

x_normalized

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [3]:
y = pd.get_dummies(df.iloc[:,-1], prefix='y')

y

,y_0,y_1,y_2,y_3
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
1995,1,0,0,0
1996,0,0,1,0
1997,0,0,0,1
1998,1,0,0,0


In [4]:
df_normalized = pd.concat([x_normalized, df['price_range']], axis=1)

df_normalized

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,0.227789,0.0,0.68,0.0,0.052632,0.0,0.080645,0.555556,0.900000,0.142857,...,0.010204,0.170895,0.612774,0.285714,0.388889,0.944444,0.0,0.0,1.0,1
1,0.347361,1.0,0.00,1.0,0.000000,1.0,0.822581,0.666667,0.466667,0.285714,...,0.461735,0.993324,0.634687,0.857143,0.166667,0.277778,1.0,1.0,0.0,2
2,0.041416,1.0,0.00,1.0,0.105263,1.0,0.629032,0.888889,0.541667,0.571429,...,0.644388,0.811749,0.627205,0.428571,0.111111,0.388889,1.0,1.0,0.0,2
3,0.076152,1.0,0.80,0.0,0.000000,0.0,0.129032,0.777778,0.425000,0.714286,...,0.620408,0.858478,0.671566,0.785714,0.444444,0.500000,1.0,0.0,0.0,2
4,0.881764,1.0,0.28,0.0,0.684211,1.0,0.677419,0.555556,0.508333,0.142857,...,0.616327,0.475300,0.308658,0.214286,0.111111,0.722222,1.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.195725,1.0,0.00,1.0,0.000000,1.0,0.000000,0.777778,0.216667,0.714286,...,0.623469,0.927904,0.110102,0.571429,0.222222,0.944444,1.0,1.0,0.0,0
1996,0.977956,1.0,0.84,1.0,0.000000,0.0,0.596774,0.111111,0.891667,0.428571,...,0.466837,0.977971,0.474613,0.428571,0.555556,0.777778,1.0,1.0,1.0,2
1997,0.941884,0.0,0.16,1.0,0.052632,1.0,0.548387,0.666667,0.233333,1.000000,...,0.442857,0.755674,0.748530,0.285714,0.055556,0.166667,1.0,1.0,0.0,3
1998,0.675351,0.0,0.16,0.0,0.210526,1.0,0.709677,0.000000,0.541667,0.571429,...,0.171429,0.113485,0.163816,0.928571,0.555556,0.944444,1.0,1.0,1.0,0


In [5]:
class Autoencoder(nn.Module):
    def __init__(self, layers=[], h_activation="relu", o_activation="sigmoid", device=torch.device("cpu")):
        super().__init__()

        self.device = device

        self.h_activation = h_activation
        self.o_activation = o_activation

        self.encoding_layers = nn.ModuleList([])
        self.decoding_layers = nn.ModuleList([])

        self.layers = layers

        reversed_layers = list(reversed(layers))

        for i in range(len(layers) - 1):
            self.encoding_layers.append(nn.Linear(layers[i], layers[i+1]))
            self.decoding_layers.append(nn.Linear(reversed_layers[i], reversed_layers[i+1]))

        # Initialize model to device
        self.to(self.device)

    def encode(self, x):
        for i in range(len(self.encoding_layers)):
            if self.h_activation == "relu":
                x = F.relu(self.encoding_layers[i](x))
            else:
                raise Exception("Invalid hidden activation {}".format(self.h_activation))

        return x

    def decode(self, x):
        for i in range(len(self.decoding_layers)):
            if i != len(self.decoding_layers) - 1:
                if self.h_activation == "relu":
                    x = F.relu(self.decoding_layers[i](x))
                else:
                    raise Exception("Invalid hidden activation {}".format(self.h_activation))
            else:
                if self.o_activation == "sigmoid":
                    x = torch.sigmoid(self.decoding_layers[i](x))
                else:
                    raise Exception("Invalid output activation {}".format(self.o_activation))

        return x

    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)

        return x

    def load(self, filename):
        state = torch.load(filename)

        self.load_state_dict(state['state_dict'])

        self.optimizer = state['optimizer']

        params = state.get('params')

        self.o_activation   = params['o_activation']
        self.h_activation   = params['h_activation']
        self.layers         = params['layers']
        self.device         = params['device']

In [6]:
class AutoencoderDataset(Dataset):         
    def __init__(self, x):    
        self.x = x

        self.n_samples = len(x)
                                                                  
    def __getitem__(self, index):                   
        return self.x[index], self.x[index]                
                                                     
    def __len__(self):                                                                                   
        return self.n_samples

In [14]:
layers = [20, 10, 2]

model = Autoencoder(       
    layers=layers
)

model

Autoencoder(
  (encoding_layers): ModuleList(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=2, bias=True)
  )
  (decoding_layers): ModuleList(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=20, bias=True)
  )
)

In [15]:
learning_rate = 0.001
loss_fn = nn.MSELoss()
epochs = 100
batch_size = 5
device = 'cpu'

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
x = torch.tensor(x_normalized.values).float().to(device)

train_ds = AutoencoderDataset(
    x=x
)

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader)

    ave_loss = 0.0
    count = 0

    for batch_idx, (data, targets) in enumerate(loop):
        data    = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        predictions = model.forward(data)

        loss = loss_fn(predictions, targets)

        # Backward
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        # update tqdm
        loop.set_postfix(loss=loss.item())

        ave_loss += loss.item()
        count += 1

    ave_loss = ave_loss / count

    return ave_loss

In [18]:
losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(train_loader, model, optimizer, loss_fn)
    print("Ave Loss: {}".format(ave_loss))

    state = {
        'params': {
            'o_activation': model.o_activation,
            'h_activation': model.h_activation,
            'layers':       model.layers,
            'device':       model.device
        },
        'state_dict':       model.state_dict(),
        'optimizer':        optimizer.state_dict()
    }
    
    torch.save(state, "autoencoder.pth")
    
    losses.append(ave_loss)

Epoch: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 902.31it/s, loss=0.122]


Ave Loss: 0.12558962449431418
Epoch: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 819.42it/s, loss=0.118]


Ave Loss: 0.1200458045117557
Epoch: 2


...

Ave Loss: 0.09872677028179169
Epoch: 98


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 789.08it/s, loss=0.103]


Ave Loss: 0.09844876123592257
Epoch: 99


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 837.64it/s, loss=0.103]


Ave Loss: 0.09866326093673707


In [19]:
z = model.encode(x)

z

tensor([[0.0000, 7.6915],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        ...,
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [20]:

compressed = TSNE(
    n_components=2, 
    learning_rate='auto',
    init='random'
).fit_transform(x_normalized.values)

In [21]:
compressed

array([[-33.04666  ,   5.17679  ],
       [ 49.401123 ,  14.222052 ],
       [ 49.383083 ,  14.489789 ],
       ...,
       [ 32.895844 ,  28.762457 ],
       [  3.8339033,   1.2385592],
       [ 28.77293  , -14.3853   ]], dtype=float32)

In [29]:
df_compressed = pd.DataFrame(z.detach().cpu().numpy())

df_compressed.columns = ['x1', 'x2']

df_compressed = pd.concat([df_compressed, df['price_range']], axis=1)

df_compressed

,x1,x2,price_range
0,0.0,7.691494,1
1,0.0,0.000000,2
2,0.0,0.000000,2
3,0.0,4.650342,2
4,0.0,0.000000,1
...,...,...,...
1995,0.0,0.000000,0
1996,0.0,2.428352,2
1997,0.0,0.000000,3
1998,0.0,0.000000,0


In [30]:
fig = px.scatter(df_compressed, x="x1", y="x2", color="price_range")
fig.show()